# Inference Example

#### Table of Contents
<a id='toc'></a>
- [Setup](#setup)
- [Imputation](#imputation)
- [Create Features/Labels](#createfeatures)
- [Logistic Regression](#logreg)
- [Machine Learning](#machinelearning)
- [Adding Group Membership](#groupmember)
    - [Logistic with Group Feature](#grouplog)
    - [ML with Group Features](#groupml)


During the lecture today we discussed different factors that can affect inference.  As a result of this notebook you will learn how a "grouping" variable can impact the fit of a model.  We'll also be reviewing some of the ML code you saw during the last session.

The dataset used in this notebook is ada_class3.for_inference_example and was developed for this purpose.  It is a subset of household benefit spells reported as ending within 2013.  There are over 200k rows in the dataset.

Our goal with this analysis is to predict individuals returning to benefits within 1 year after the end of a benefit spell.  Variables that will be used in this analysis are listed below.  The majority of the features will be what we will consider "individual variables," and we'll consider the *district* variable a group variable.  We will first run models with the individual features, and then see if the inclusion of the group variable changes our model output/prediction.

We will first look at the more familiar Logistic Regression, then at ML models we learned in the last class session.

## Important Variables
---
### Identification Variables
* receptno = IDHS provided receipt number
* ch_dpa_caseid = Chapin Hall Case ID number
* new_id = unique row id number created for this dataset
* start_date
* end_date

### Features/Grouping Variables
*note that variables from the case records may have missingness*

* benefit_type 
* sex
* rac
* rootrace
* foreignbrn
* edlevel
* health
* martlst
* workexp
* district
* homeless

#### Features Developed From Wage Table
* has_job_win1yr = 0/1 indicating any wage table employment within one year
* lose_job_win1yr = 0/1 indicating that had wage table employment and then did not within one year
* has_job_q(1-4) = 0/1 variable for each of the 4 quarters in the year after end_date indicating employment
* wage_q(1-4) = wage total for each quarter (could be from multiple jobs)
* total_wage_1yr = sum of all the wages within the year following end_date

### Labels/Outcome Variables
* new_spell_win1yr = 0/1 variable indicating a new spell within one year of end date of spell
* new_spell_win1yr_benefit = same as above, but has to be same benefit_type (tanf, foodstamps, etc.)

# Setup
<a id='setup'></a>
- Return to [Table of Contents](#toc)

In [ ]:
%pylab inline
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras
import sklearn
import seaborn as sns
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              GradientBoostingClassifier,
                              AdaBoostClassifier)
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
import sqlalchemy
sns.set_style("white")

In [ ]:
db_name = "appliedda"
hostname = "10.10.2.10"
conn = psycopg2.connect(database=db_name, host = hostname) #database connection


In [ ]:
#load some of Avishek's function defintions to help with model comparison
def plot_precision_recall_n(y_true, y_prob, model_name):
    """
    y_true: ls 
        ls of ground truth labels
    y_prob: ls
        ls of predic proba from model
    model_name: str
        str of model name (e.g, LR_123)
    """
    from sklearn.metrics import precision_recall_curve
    y_score = y_prob
    precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_true, y_score)
    precision_curve = precision_curve[:-1]
    recall_curve = recall_curve[:-1]
    pct_above_per_thresh = []
    number_scored = len(y_score)
    for value in pr_thresholds:
        num_above_thresh = len(y_score[y_score>=value])
        pct_above_thresh = num_above_thresh / float(number_scored)
        pct_above_per_thresh.append(pct_above_thresh)
    pct_above_per_thresh = np.array(pct_above_per_thresh)
    plt.clf()
    fig, ax1 = plt.subplots()
    ax1.plot(pct_above_per_thresh, precision_curve, 'b')
    ax1.set_xlabel('percent of population')
    ax1.set_ylabel('precision', color='b')
    ax1.set_ylim(0,1.05)
    ax2 = ax1.twinx()
    ax2.plot(pct_above_per_thresh, recall_curve, 'r')
    ax2.set_ylabel('recall', color='r')
    ax2.set_ylim(0,1.05)
    
    name = model_name
    plt.title(name)
    plt.show()
    plt.clf()
    
def precision_at_k(y_true, y_scores,k):
    
    threshold = np.sort(y_scores)[::-1][int(k*len(y_scores))]
    y_pred = np.asarray([1 if i >= threshold else 0 for i in y_scores ])
    return precision_score(y_true, y_pred)

In [ ]:
select_statement = """SELECT new_id, sex, rootrace, edlevel, workexp, martlst, homeless, benefit_type,
                    has_job_q1, has_job_q2, has_job_q3, has_job_q4, wage_q1, wage_q2, wage_q3, wage_q4,
                    has_job_win1yr, lose_job_win1yr, total_wage_1yr, new_spell_win1yr, new_spell_win1yr_benefit,
                    district FROM ada_class3.for_inference_example WHERE total_wage_1yr IS NOT NULL;"""
df = pd.read_sql( select_statement, conn )
print df.shape
#df.head()

## Imputation
<a id='imputation'></a>
- Return to [Table of Contents](#toc)

We can see from just the few rows above that there is clearly missing data.  Because our variables are categorical, we cannot simply impute the mean for the missing values.  Instead, we will add a 0/1 variable for missing in each variable that will be another feature/predictor used in the model.

In [ ]:
df['sex_miss'] = (df['sex'] == 0)
df['race_miss'] = (df['rootrace'] == 0)
df['ed_miss'] = (df['edlevel'] == None)
df['mar_miss'] = (df['martlst'] == 0)
df['home_miss'] = (df['homeless'] == None)
#df.head()

## Create Features
<a id='createfeatures'></a>
- Return to [Table of Contents](#toc)

We need to create 0/1 "dummy" variables/features for the rest of the different levels of the predictors we want to include in our model. At the end of this code block we set up two different feature tables - one with the yearly employment features and the other with the quarterly employment features.

In [ ]:
#sex
df['male'] = (df['sex'] == 1)
df['female'] = (df['sex'] == 2)
#rootrace
df['nhwhite'] = (df['rootrace'] == 1)
df['nhblack'] = (df['rootrace'] == 2)
df['native'] = (df['rootrace'] == 3)
df['hispanic'] = (df['rootrace'] == 6)
df['asian'] = (df['rootrace'] == 7)
#edlevel
less_list = ['A', 'B', 'C', 'D', 1, 2, 3]
somehs_list = ['E', 'F', 4]
hsgrad_list = ['G', 'H', 'V', 5]
somecoll_list = ['W', 'X', 'Y', 6]
collgrad_list = ['Z', 'P', 7]
df['lessthanhs'] = (df['edlevel'].isin(less_list))
df['somehs'] = (df['edlevel'].isin(somehs_list))
df['hsgrad'] = (df['edlevel'].isin(hsgrad_list))
df['somecoll'] = (df['edlevel'].isin(somecoll_list))
df['collgrad'] = (df['edlevel'].isin(collgrad_list))
#workexp
df['noattach'] = (df['workexp'] == 0)
df['nowkexp'] = (df['workexp'] == 1)
df['prof'] = (df['workexp'] == 2)
df['othermgr'] = (df['workexp'] == 3)
df['clerical'] = (df['workexp'] == 4)
df['sales'] = (df['workexp'] == 5)
df['crafts'] = (df['workexp'] == 6)
df['oper'] = (df['workexp'] == 7)
df['service'] = (df['workexp'] == 8)
df['labor'] = (df['workexp'] == 9)
#martlst
df['nvrmar'] = (df['martlst'] == 1)
df['marwspouse'] = (df['martlst'] == 2)
df['marwospouse'] = (df['martlst'].isin([3,4,6]))
df['sepordiv'] = (df['martlst'].isin([5,7]))
df['widow'] = (df['martlst'] == 8)
#homeless
df['nothomeless'] = (df['homeless'] == 'N')
df['ishomeless'] = (df['homeless'].isin(['1','2','3','4','Y']))
#benefit_type
df['foodstamp'] = (df['benefit_type'] == 'foodstamp')
df['tanf'] = (df['benefit_type'] == 'tanf46')
df['grant'] = (df['benefit_type'] == 'grant')
#create features df

df_features = df[['male', 'female', 'nhwhite', 'nhblack', 'native', 'hispanic', 'asian', 'lessthanhs', 
                  'somehs', 'hsgrad', 'somecoll', 'collgrad', 'noattach', 'nowkexp', 'prof', 'othermgr',
                  'clerical', 'sales', 'crafts', 'oper', 'service', 'labor', 'nvrmar', 'marwspouse', 
                  'sepordiv', 'widow', 'nothomeless', 'ishomeless', 'foodstamp', 'tanf', 'grant']].copy()
# features df with qtr based job variables
df_features_wjobqtr = df[['has_job_q1', 'has_job_q2', 'has_job_q3', 'has_job_q4',
                'wage_q1', 'wage_q2', 'wage_q3', 'wage_q4']].copy()
df_features_wjobqtr.join(df_features) 
# features df with year based job variables
df_features_wjobyr = df[['has_job_win1yr', 'lose_job_win1yr', 'total_wage_1yr']].copy()
df_features_wjobyr = df_features_wjobyr.join(df_features)

In [ ]:
#print frequencies of dummy variables - for disclosure review.
feat_list = list(df_features)
for item in feat_list:
    print pd.crosstab(index=df_features[item], columns = "count")


### Create Labels
The labels dataframes each have just one column, the outcome variable for your model.  Below we've created two, one for returns to any benefit within the year after a spell, the other is an indicator for return to the same benefit within the year.  We'll be using the first in the examples below.

In [ ]:
df_label_returnany = df[['new_spell_win1yr']].copy()
df_label_returnsame = df[['new_spell_win1yr_benefit']].copy()

### Start with a more familiar model - Logistic Regression
<a id='logreg'></a>
- Return to [Table of Contents](#toc)

We'll first fit a logistic regression model to predict return to benefits based on the individual predictors we have selected.  Typically when a social scientist uses this type of model it is to describe the individual predictors' effects on some outcome, therefore the entire dataset is used to fit the model.

Unlike in the ML class, we will be using the package *statsmodels* to run our Logistic Regression models, since it provides many more output options/functions vs. sci-kit learn. 

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# create one df and set up listings of variables for below
#drop reference categories (missing, male)
df_features_forlr = df_features_wjobyr[['female', 'nhwhite', 'nhblack', 'native', 'hispanic', 'asian', 'lessthanhs',
                                        'somehs', 'hsgrad', 'somecoll', 'collgrad', 'noattach', 'nowkexp', 'prof', 'othermgr', 
                                        'clerical', 'sales', 'crafts', 'oper', 'service', 'labor', 'nvrmar', 'marwspouse', 
                                        'sepordiv', 'widow', 'nothomeless', 'ishomeless', 'foodstamp', 'tanf', 'grant', 
                                        'has_job_win1yr', 'lose_job_win1yr', 'total_wage_1yr']].copy()
df_lrmodel = df_features_forlr.join(df_label_returnany)

*statsmodels* takes as an argument a string with your model specification, in a format you may be familiar with if you have used R in the past.  In order to build that list I'll employ some code to avoid typing all the variable names.

In [ ]:
#get list of features to build model statement
feat_list = list(df_features_forlr)
print feat_list
print len(feat_list)
length = len(feat_list)

#create string of features with plus signs
count = 0
feat_string = ''
for feature in feat_list:
    count += 1
    if count < (length):
        feat_string += feature 
        feat_string += ' + '
    else:
        feat_string += feature
## END FOR BLOCK

print feat_string

In [ ]:
formula = "new_spell_win1yr ~ " + feat_string
print (formula)

In [ ]:
#fit model - note the procedure is glm so you have to specify binomial in the family argument to get LR
model =smf.glm(formula=formula, data=df_lrmodel, family=sm.families.Binomial())
result = model.fit()
print (result.summary())

It's no surprise that the majority of the coefficients are significant - with Ns this large significance becomes pretty meaningless.  Looking at the direction of the coefficients, we start to see which variables impact the probability of going back on benefits either positive or negatively.  For female and nhblack (Non-Hispanic Blacks) we see they are more likely to return to benefits (reference values are male and missing-race).  On the other hand nhwhite (Non-Hispanic Whites) are less likely to return.

This package also has the benefit of returning a model output summary familiar to any SAS or stata user.  We can also get just the values using .params or .pvalues (for example result.params contains just the coefficients for this model

In [ ]:
print (result.params)

However, we have run this model how a statistician would typically - using all of the data to fit the model.  In our ML notebook we learned about using training and test sets of data to show the strength of the model in predicting a subsequent outcome.  This can also be done with the logistic regression as we saw in the ML notebook.  Below we'll work through it in *statsmodels* with our same data from above.  So we can cut our dataframe into train and test sets.  Here I'll use the index (row numbers) to cut the dataframe into roughly 80/20 train/test.

In [ ]:
df_lrtrain = df_lrmodel[:201540]
df_lrtest = df_lrmodel[201540:]
print df_lrtrain.shape
print df_lrtest.shape

In [ ]:
#fit model - this time we're only fitting the model to the training set.
model =smf.glm(formula=formula, data=df_lrtrain, family=sm.families.Binomial())
result = model.fit()
print (result.summary())

This is fitting the model to just 80% of the data that we used above, however, as to be expected, the resulting coefficients are largely similar.  

Compared to the first model, we get:

XXX

Any coefficients with larger changes between this model (like native which went from XXX) is because we cut data out of our dataset in a not exactly random way and those are rarer instances.

Now we can use these coefficients to make predictions on the test set and see how well our model works for prediction.

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

predictions = result.predict(df_lrtest)

pred_binary = (predictions > 0.5)

print ("confusion matrix")
print confusion_matrix(df_lrtest['new_spell_win1yr'], pred_binary )
print classification_report(df_lrtest['new_spell_win1yr'], pred_binary, digits=3 )

plot_precision_recall_n(df_lrtest['new_spell_win1yr'], predictions, "Logistic Regression")

## Moving to Machine Learning
<a id='machinelearning'></a>
- Return to [Table of Contents](#toc)

### Create Testing/Training sets of both Features and Labels sets
We're going to start with using the features table with the yearly job variables and the label that indicates a return to any form of benefits.

This is the same code as we used in the ML notebook.

In [ ]:
# create train/test sets
X_train, X_test, y_train, y_test = train_test_split(df_features_wjobyr, df_label_returnany, test_size = 0.2)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape
sel_features = list(X_train)

### Running through the ML models
Below is again code familiar from the ML notebook.  We set up our arguments in a dictionary so that we can loop through the models and print the results using the same code.

Again we're using the full set of features to predict returning to benefits within 1 year of the end of a spell.

In [ ]:
clfs = {'RF': RandomForestClassifier(n_estimators=50, n_jobs=-1),
       'ET': ExtraTreesClassifier(n_estimators=10, n_jobs=-1, criterion='entropy'),
        'LR': LogisticRegression(penalty='l1', C=1e5),
        'SGD':SGDClassifier(loss='log'),
        'GB': GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, random_state=17, n_estimators=10),
        'NB': GaussianNB()}

In [ ]:
sel_clfs = ['RF', 'ET', 'LR', 'SGD', 'GB', 'NB']

In [ ]:
max_p_at_k = 0
for clfNM in sel_clfs:
    clf = clfs[clfNM]
    clf.fit( X_train, y_train.values.ravel() )
    print clf
    y_score = clf.predict_proba(X_test)[:,1]
    predicted = np.array(y_score)
    expected = np.array(y_test)
    plot_precision_recall_n(expected,predicted, clfNM)
    p_at_1 = precision_at_k(expected,y_score, 0.05)  ## note that i changed the k value here from 0.01 to 0.05
    print('Precision at 5%: {:.2f}'.format(p_at_1))

To summarize, the Precision at %5 values for the 6 models are:

* XXXX

To get the best prediction for our money, we'll probably want to choose a model with high precision at 5% (the level of funding we can afford). 

Focusing on the ML models with highest precision at %5, we'll look at the feature importances for these two ML models (Random Forest and Gradient Boosting).

Feature importances indicate which variables are most important to the prediction/classification of cases as the trees split.  Features at earlier/higher splits are more important than those at lower levels in the tree.

In [ ]:
#select only the two ML models
sel_clfs = ['RF', 'GB']

In [ ]:
#here I've adapted the model loop from above to print feature importances instead of the precision/recall graph

for clfNM in sel_clfs:
    clf = clfs[clfNM]
    clf.fit( X_train, y_train.values.ravel() )
    print clf
    y_score = clf.predict_proba(X_test)[:,1]
    predicted = np.array(y_score)
    expected = np.array(y_test)
    
    var_names = list(X_train) # get a list of variable names
        
    importances = clf.feature_importances_ # get the feature importances
    indices = np.argsort(importances)[::-1] # sort the list to get the highest importance first
    
    for f in range(X_train.shape[1]):
        print ("%d. feature (%s) importance = %f" % (f + 1, var_names[indices[f]], importances[indices[f]]))    
    

    p_at_1 = precision_at_k(expected,y_score, 0.05)
    print('Precision at 5%: {:.2f}'.format(p_at_1))
    print

The most important features in the Random Forest Model are:
1. total_wage_1yr 
2. nothomeless 
3. nvrmar
4. hsgrad
5. noattach

This makes sense - wages and homelessness status likely have a huge impact on whether a person returns to benefits within the year or not.

The most important features in the Gradient Boosting Model are:
1. nothomeless
2. nvrmar
3. total_wage_1yr
4. foodstamp
5. hsgrad

Again we see most of the same variables.  The slight differences are due to the different ways in which these models are processing the same data.  

# Will Adding a Group Membership Variable make a difference?
<a id='groupmember'></a>
- Return to [Table of Contents](#toc)

We'll add in the district variable and see if it makes a difference in either our prediction or our coefficients/feature importances.  Adding this variable could improve prediction to the extent that it is new information not already contained within the other features.  It can also change the coefficients in a logistic model or the feature importances in a ML model if there are clusters of individuals in the groups who are alike on that particular feature.  For example, if there are more individuals of a particular race collected in one of the groups, not including the group variable meant that the race variable was partially serving as "proxy" for the group membership.  Adding the group membership to the model lets you see the effect of race above and beyond the clustering.

We will use the *district* variable and split it into two groupings - downstate (codes 10-115) and Cook County (codes 200-294). 


In [ ]:
df['cookcty'] = ((df['district'] >= 200) & (df['district'] <= 294))
df['downstate'] = ((df['district'] >= 10) & (df['district'] <= 115))

### Return to Logistic Regression
<a id='grouplog'></a>
- Return to [Table of Contents](#toc)

Adding in our new variable.

In [ ]:
# add new cookcty to df for logistic regression
df_lrmodel['cookcty'] = df['cookcty']
df_lrtrain = df_lrmodel[:201540]
df_lrtest = df_lrmodel[201540:]
print df_lrtrain.shape
print df_lrtest.shape

In [ ]:
formula += " + cookcty" 
model =smf.glm(formula=formula, data=df_lrtrain, family=sm.families.Binomial())
result = model.fit()
print (result.summary())

Compared to our previous LR model, we get:

XXX

So adding the group variable did account for some of the race effects we were seeing in the previous model.

We also see cookcty has a coefficient of XXX, reflecting the power that group membership has on the prediction

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

predictions = result.predict(df_lrtest)

pred_binary = (predictions > 0.5)


print confusion_matrix(df_lrtest['new_spell_win1yr'], pred_binary )
print classification_report(df_lrtest['new_spell_win1yr'], pred_binary, digits=3 )

plot_precision_recall_n(df_lrtest['new_spell_win1yr'], predictions, "Logistic Regression")

The slight improvement to the precision is reflected in the graph above.

### Will ML Models be changed?
<a id='groupml'></a>
- Return to [Table of Contents](#toc)

We saw an impact of adding this grouping variable in the Logistic Regression.  Will it improve the predictive power of our ML models?  Will it be an important feature?

None of our model fitting code is changed below - we're just adding two new features and rerunning.

In [ ]:
df_features_wjobyr['cookcty'] = df['cookcty']
df_features_wjobyr['downstate'] =df['downstate']

X_train, X_test, y_train, y_test = train_test_split(df_features_wjobyr, df_label_returnany, test_size = 0.2)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape
sel_features = list(X_train)

In [ ]:
sel_clfs = ['RF', 'ET', 'LR', 'SGD', 'GB', 'NB']
max_p_at_k = 0
for clfNM in sel_clfs:
    clf = clfs[clfNM]
    clf.fit( X_train, y_train.values.ravel() )
    print clf
    y_score = clf.predict_proba(X_test)[:,1]
    predicted = np.array(y_score)
    expected = np.array(y_test)
    plot_precision_recall_n(expected,predicted, clfNM)
    p_at_1 = precision_at_k(expected,y_score, 0.05)
    print('Precision at 5%: {:.2f}'.format(p_at_1))

The Precision at %5 did not substantially improve with any of the models.  And the prediction power of the SGD model was decreased.

* XXX

We'll again look at the feature importances to see if there are any substantial changes.

In [ ]:
sel_clfs = ['RF', 'GB']
for clfNM in sel_clfs:
    clf = clfs[clfNM]
    clf.fit( X_train, y_train.values.ravel() )
    print clf
    y_score = clf.predict_proba(X_test)[:,1]
    predicted = np.array(y_score)
    expected = np.array(y_test)
    
    var_names = list(X_train)
        
    importances = clf.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    for f in range(X_train.shape[1]):
        print ("%d. feature (%s) importance = %f" % (f + 1, var_names[indices[f]], importances[indices[f]]))    
    

    p_at_1 = precision_at_k(expected,y_score, 0.05)
    print('Precision at 5%: {:.2f}'.format(p_at_1))
    print

Our most important features in each model are largely unchanged.  *cookcty* and *downstate* show up as 16th and 21st most important features in the Random Forest model, respectively.  In the Gradient Boosing model, though, *cookcty* shows up 5th, indicating it was an important classifier, however the resulting output slightly lowered our precision at 5%.